<h1 style="color:green;">Design patterns for option pricing via simulation<h1>
    <blockquote style="font-family:Calibri; font-size:16px;">This is a personal project of mine, investigating how to price different financial options through simulation and, where possible comparing these values to analytic prices</blockquote>

<h2 style="color:green;">Part 0: Setup-<h2>
<blockquote style="font-family:Calibri; font-size:16px;">Import libraries, establish and justify RNG choice, numerous random draws from a Gaussian distribution will form the cornerstone of our method, we find random.gauss to produce results more quickly </blockquote>

In [2]:
import time 
import random 
import numpy
import math
import scipy.integrate as integrate #integration kit
from scipy.stats import norm
import statistics

tic=time.perf_counter()
for i in range(10**6):
    random.gauss(0,1)
toc=time.perf_counter()

tic2=time.perf_counter()
for i in range(10**6):
    numpy.random.normal()
toc2=time.perf_counter()
print("\nRunning time random = {} ms".format(int((toc-tic)*10**3))+
      "\nRunning time np.normal = {} ms".format(int((toc2-tic2)*10**3)))
#we use random.gauss due to it's ability to generate normal draws more quickly!




Running time random = 1057 ms
Running time np.normal = 2898 ms


In [3]:
class simulationStart:
    def __init__(self,spot,sims,vol,steps,t,mu,strike):
        self.S0=spot
        self.sims=sims
        self.vol=vol
        self.steps=steps
        self.strike=strike
        self.t=t
        
        self.mu=mu

    def lowMemorySimulations(self): #good for calculating EV without using memory (arr)- no descriptive stats obtainable
        totsum=0
        sqrtt=math.sqrt(self.t)
        for j in range(self.sims):
            Spot=self.S0
            for i in range(self.steps):
                N01=random.gauss(0,1)
                Spot=Spot*(1+(self.mu*self.t+self.vol*sqrtt*N01))  #in this discrete approximate model we can go negative!!
            totsum+=Spot #memoryless- does not require storing an array

        return totsum/self.sims 
    def simulationsArray(self): #keep array
        arr=[]
        sqrtt=math.sqrt(self.t)
        for j in range(self.sims):
            Spot=self.S0
            for i in range(self.steps):
                N01=random.gauss(0,1)
                Spot=Spot*(1+(self.mu*self.t+self.vol*sqrtt*N01))  #in this discrete approximate model we can go negative!!
            arr.append(Spot) #keep an array
        return arr



            





<h2 style="color:green;">Section 1: Pricing using log normal assumptions<h2>
<blockquote style="font-family:Calibri; font-size:16px;">Build progressvely more complex options, compare analytic price to simulations </blockquote>

In [3]:
sm=simulationStart(spot=100,sims=10**3,vol=0.1,steps=365,mu=0.05,t=1/365,strike=105) #Stockmotion=sm


mean=sm.lowMemorySimulations()

#dS=mu*S*dt+sigma*dWt


#Analytical mean = S*exp(mu*t)
#Derived from integral of S0*e^(mu*t-0.5*sigma^2+sigma*x)*e^-(x^2/2)
print("\n Mean =" +format(mean))
print("\n Analytic integration Expected value =" + format(sm.S0*math.exp(sm.mu*sm.t*sm.steps)))

#Numerical integration method
result = integrate.quad(lambda x: (1/math.sqrt(2*math.pi))*sm.S0*math.exp(sm.mu*sm.t*sm.steps-0.5*sm.vol**2+sm.vol*x)*math.exp(-(x**2)/2), -10, 10) #+/-10 s.d.s sufficiently converged for double floating point precision
print("\n Numerical integration EV & error bounds =" + format(result))


 Mean =105.10272582673365

 Analytic integration Expected value =105.12710963760242

 Numerical integration EV & error bounds =(105.12710963760242, 1.0679979662274464e-07)


<blockquote style="font-family:Calibri; font-size:16px;"> Here we stochastically simulate 10,000 runs of a stock with spot 100 and annualised drift 0.05. Via analytical integration, numerical integration and Monte Carlo simulation we find the final prices agree. This type of stochastic simulation will be key to our first section (later we will investigate other volatility simulations (e.g. time and price dependent) </blockquote>

<blockquote style="font-family:Calibri; font-size:16px;"> We now begin Section one in earnest, starting with attempting to price a forward contact, progressively moving to more complex derivatives. Note: We will not always have tidy integration to help us!

<h3 style="color:green;">Fwd contract<h3>
</blockquote>

In [23]:
#A forward contract is the right and the obligation to buy a stock at time t for price K.
#The payoff is S-K.
#Using risk neutral pricing E(C_T/Z_T)=C_0/Z_0 
#Our only unknown in this equaton is C_0.
#C_T=S-K
#Z_0=e^-rT
#Z_T=1
#C_0=(e^-rT)*E[(S-K)]
#S_0
sm2=simulationStart(spot=100,sims=10**3,vol=0.1,steps=365,mu=0.05,t=1/365,strike=95) #Stockmotion=sm

arr=sm2.simulationsArray()

# for j in range(sims):
#     Spot=S0
#     for i in range(Steps):
#         N01=random.gauss(0,1)
#         Spot=Spot*(1+(mu*t+vol*sqrtt*N01))  #in this discrete approximate model we can go negative!!
#     arr.append(Spot-strike) 


#Find e^-rtE(S-K)- K is const/ not a RV, K=strike=95
rt=sm2.mu*sm2.t*sm2.steps #mu=rt in risk neutral world

print("\n Sample Mean =" +format(math.exp(-rt)*statistics.mean(arr)-sm2.strike))
print("\n Expected Value =" +format(math.exp(-rt)*sm.S0*math.exp(sm2.mu*sm2.t*sm2.steps)-sm2.strike))









 Sample Mean =4.681959382095172

 Expected Value =5.000000000000014


<blockquote style="font-family:Calibri; font-size:16px;">The simulation method for forward contract is identical, simulate the stock as necessary- then plug each value into the payoff function (S-K)</blockquote>

<h3 style="color:green;">Call option<h3>
<blockquote style="font-family:Calibri; font-size:16px;">The vanilla call option, with pay off (S-K) can be solved analytically in at least two ways- by solving the Black-Scholes eqn and via risk-neutral pricing, use riskless bond for numeraire <br> 
    $dS=\mu Sdt+\sigma SdW_{t}$ <br>
    $dB=rBdt$ <br>
    
Risk-neutral pricing requires $d(S/B)$ be a Martingale. Using Ito's lemma we get: <br>
    $d(S/B)=(\mu-r)\frac S B dt+randomness$, Martingale $ \iff \mu=r$ <br>
Thus using RN pricing:
    $\frac {C_0} {B_0}=\mathbb E[\frac {C_T}{ B_T}]$ <br>
Equivalently $C_0=e^{-rt}E[{C_T} ]$
We may now proceed with both MC and probabilistic method
 </blockquote>

In [62]:
sm3=simulationStart(spot=100,sims=10**3,vol=0.1,steps=365,mu=0.05,t=1/365,strike=95) #Stockmotion=sm
arr=sm3.simulationsArray()

# for j in range(sims):
#     Spot=S0
#     for i in range(Steps):
#         N01=random.gauss(0,1)
#         Spot=Spot*(1+(mu*t+vol*sqrtt*N01))  #in this discrete approximate model we can go negative!!
#     arr.append(Spot-strike) 


#Find e^-rtE(S-K)- K is const/ not a RV, K=strike=95
rt=sm3.mu*sm3.t*sm3.steps #mu=rt in risk neutral world
for i in range(len(arr)):
    arr[i]=max(arr[i]-sm3.strike,0)
    

d1=(math.log(sm3.S0/sm3.strike)+(sm3.mu+0.5*sm3.vol**2)*sm3.t*sm3.steps)/(sm3.vol*math.sqrt(sm3.t*sm3.steps))
d2=(math.log(sm3.S0/sm3.strike)+(sm3.mu-0.5*sm3.vol**2)*sm3.t*sm3.steps)/(sm3.vol*math.sqrt(sm3.t*sm3.steps))

print("\n Sample Mean =" +format(math.exp(-rt)*(statistics.mean(arr))))
print("\n Expected Value" + format(sm3.S0*norm.cdf(d1)-sm3.strike*math.exp(-rt)*norm.cdf(d2)))





 Sample Mean =10.322783140874224

 Expected Value10.405284289598569


<blockquote style="font-family:Calibri; font-size:16px;"> Here we stochastically simulate 10,000 runs of a stock with spot 100 and annualised drift 0.05. Via analytical integration, numerical integration and Monte Carlo simulation we find the final prices agree. This type of stochastic simulation will be key to our first section (later we will investigate other volatility simulations (e.g. time and price dependent) </blockquote>


<blockquote style="font-family:Calibri; font-size:16px;"> Barrier option: Barrier options can knock in or knock out when they reach a certain value- they can be analytically priced via the reflection principle and Girsanov's theorem. We price a down and in option. The other barrier prices are similarly derived. </blockquote>

For a down and out barrier struck at y with barrier x, an option has payout iff $\mathbb P (W_{t}>y, m_{t}>x)$. We need to compute this probability. Via the reflection principle we have: $\mathbb P (W_{t}>y, m_{t}>x)=1-\mathbb P (W_{t}>y, m_{t}<=x)=1-\mathbb P (W_{t}<2y-x)$, the analytic price is: $S_{0}N(d_{1})-Ke^{-rT}-(\frac{H}{S_{0}})^{1+2r\sigma^-2}S_{0}N(h_{1})+(\frac{H}{S_{0}})^{-1+2r\sigma^-2}Ke^{-rT}$


In [70]:
print("\n Simulation approach")
sm4=simulationStart(spot=100,sims=10**3,vol=0.1,steps=365,mu=0.05,t=1/365,strike=95) #Stockmotion=sm
spot=100
sims=10**3
vol=0.1
steps=3650 #we discretise further due to barrier condition!!
mu=0.05
t=1/3650
period=t*steps
strike=95
barrier=90

#In this case we need to estimate whether we hit the barrier at any point, one issue with this is with discrete time is-
# we may overlook hitting the barrier at high resolutions (in our case, 1 day)
#This is an area where time complexity dimensionality begins to creep in.
 
tic=time.perf_counter()    
arr=[]
sqrtt=math.sqrt(t)
for j in range(sims):
    spot=100
    flag=0
    for i in range(steps):
        N01=random.gauss(0,1)
        spot=spot*(1+(mu*t+vol*sqrtt*N01))  #in this discrete approximate model we can go negative!!
        if spot<=barrier:
            spot=0
            break
    if spot<strike:
        spot=0
    else:
        spot=spot-strike
    arr.append(spot) #keep an array
print(arr)

print("\n Sample Mean =" +format(math.exp(-mu*period)*(statistics.mean(arr))))

toc=time.perf_counter()
print("\nRunning time random = {} ms".format(int((toc-tic)*10**3)))




 Simulation approach
[23.635194339035152, 18.62876175019339, 0, 18.717563068700755, 15.775684979385105, 19.88225751740039, 0, 31.694664418927857, 0, 0, 11.128435007732321, 8.956157418230703, 22.378981298463614, 1.234017594868476, 8.021931219756695, 20.378448558617634, 27.30992226742285, 10.623971891455298, 34.62773395081078, 24.440523858892433, 11.040270139029644, 4.515801758945585, 0, 26.055875159195907, 4.797351730280312, 4.948098119097168, 15.292445630889617, 0, 29.328190880898546, 0.21994669686915813, 8.240665298049336, 11.965934607390238, 5.07190738467564, 20.172996170872352, 7.740427746851523, 0, 0, 6.965872668193327, 32.657592804526345, 0.878310128792009, 15.27031122141203, 14.392314886860206, 26.83030756602409, 30.474571384189616, 0, 0, 7.21575540908529, 1.5086529679672083, 15.04844506609308, 2.672312351344445, 4.384766971187261, 0, 15.174235593872652, 0, 0, 18.69386544983621, 11.272928334086359, 9.0290085895479, 0, 11.644318902389074, 14.569573354980278, 17.84247676575106, 7.

<blockquote style="font-family:Calibri; font-size:16px;"> Here we have used the same parameters as in the call option, we observe how the value has fallen, as expected, time complexity is also higher here, we discretise steps into 0.1 days to increase the probability barrier hits are actually accounted for. </blockquote>

In [72]:
##Analytic price
spot=100
sims=10**3
vol=0.1
steps=3650 #we discretise further due to barrier condition!!
mu=0.05
t=1/3650
period=t*steps
strike=95
barrier=90


H=barrier
d1=(math.log(spot/strike)+(mu+0.5*vol**2)*period)/(vol*math.sqrt(period))
d2=(math.log(spot/strike)+(mu-0.5*vol**2)*period)/(vol*math.sqrt(period))
h1=(math.log(H**2/(spot*strike))+(mu+0.5*vol**2)*period)/(vol*math.sqrt(period))
h2=(math.log(H**2/(spot*strike))+(mu-0.5*vol**2)*period)/(vol*math.sqrt(period))

price=spot*norm.cdf(d1)-strike*math.exp(-mu*period)*norm.cdf(d2)-(H/spot)**(1+2*mu*vol**(-2))*spot*norm.cdf(h1)+ \
(H/spot)**(-1+2*mu*vol**(-2))*strike*math.exp(-mu*period)*norm.cdf(h2)

print(price)




10.175441620958736


<blockquote style="font-family:Calibri; font-size:16px;"> Our prices match. A good benchmark. Unfortunately many SDEs under different models, and for different exotic options do not easily solve. In the next section we investigate options that can only be priced via numerical methods.  </blockquote>